In [ ]:
import pandas as pd
import numpy as np

isot_train = pd.read_csv('data/isot/preprocessed/train.csv')
isot_test = pd.read_csv('data/isot/preprocessed/test.csv')


kaggle_train = pd.read_csv('data/kaggle/preprocessed/train.csv')
kaggle_test = pd.read_csv('data/kaggle/preprocessed/test.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Fonction pour préparer les données
def prepare_data(train_df, test_df):
    # Conversion des DataFrames pandas en objets datasets
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    # Charger le tokenizer BERT
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Fonction de tokenisation
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

    # Appliquer la tokenisation aux datasets
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    return train_dataset, test_dataset, tokenizer

# Fonction pour créer et entraîner le modèle
def train_model(train_df, test_df):
    # Préparer les données
    train_dataset, test_dataset, tokenizer = prepare_data(train_df, test_df)

    # Charger le modèle pré-entraîné BERT pour la classification
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

    # Définir les arguments d'entraînement
    training_args = TrainingArguments(
        output_dir='./results',          # Dossier de sortie
        num_train_epochs=3,              # Nombre d'époques
        per_device_train_batch_size=8,   # Taille du batch pour l'entraînement
        per_device_eval_batch_size=16,   # Taille du batch pour l'évaluation
        evaluation_strategy="epoch",     # Évaluation après chaque époque
        logging_dir='./logs',            # Dossier pour les logs
    )

    # Créer le trainer
    trainer = Trainer(
        model=model,                         # Le modèle
        args=training_args,                  # Les arguments d'entraînement
        train_dataset=train_dataset,         # Dataset d'entraînement
        eval_dataset=test_dataset,           # Dataset de test
    )

    # Entraîner le modèle
    trainer.train()

    # Sauvegarder le modèle
    model.save_pretrained('./fake_news_model')
    tokenizer.save_pretrained('./fake_news_model')

    return model, tokenizer

# Fonction pour évaluer le modèle
def evaluate_model(model, tokenizer, test_df):
    # Tokenisation du test set
    test_dataset = Dataset.from_pandas(test_df)
    test_dataset = test_dataset.map(lambda x: tokenizer(x['text'], padding="max_length", truncation=True, max_length=512), batched=True)
    
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
    )
    
    results = trainer.evaluate(test_dataset)
    print(f"Évaluation sur le dataset de test: {results}")
    return results


In [12]:
# Exemple d'utilisation
if __name__ == "__main__":
    # Exemple de DataFrame train et test (remplace par tes propres données)


    # Entraînement du modèle
    model, tokenizer = train_model(kaggle_train, kaggle_test)

    # Évaluation sur le dataset de test
    evaluate_model(model, tokenizer, kaggle_test)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 1267/1267 [00:07<00:00, 176.30 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: Accelerator.__init__() got an unexpected keyword argument 'dispatch_batches'